In [1]:
import geopandas as gpd
import folium
import pandas as pd
import json
from folium.plugins import TimeSliderChoropleth
import branca

# Charger le shapefile avec GeoPandas
chemin_shapefile = "Alpes_WGS84/Alpes.shp"
gdf = gpd.read_file(chemin_shapefile)

# Vérifiez la correspondance des colonnes
df['Libelle'] = df['Libelle'].str.strip()
gdf['nom'] = gdf['nom'].str.strip()

# Joindre les données au GeoDataFrame
gdf = gdf.merge(df, left_on='nom', right_on='Libelle', how='inner')

# Centrer la carte sur les Alpes
centre_lat, centre_lon = 45.8, 6.5
carte = folium.Map(location=[centre_lat, centre_lon], zoom_start=9)

# Créer une échelle de couleurs pour l'épaisseur de neige
colormap = branca.colormap.linear.YlGnBu_09.scale(df['SNDAV'].min(), df['SNDAV'].max())

# Ajouter les massifs filtrés pour une altitude donnée
altitude_cible = 1200  # Altitude cible (modifiable)
gdf_altitude = gdf[gdf['Altitude'] == altitude_cible]

for _, row in gdf_altitude.iterrows():
    nom_massif = row['nom']
    epaisseur_neige = row['SNDAV']  # Récupérer l'épaisseur de neige
    fill_color = colormap(epaisseur_neige)  # Couleur basée sur l'épaisseur

    # Ajouter le GeoJson
    folium.GeoJson(
        row['geometry'],  # Géométrie du massif
        name=f"{nom_massif} (Altitude {altitude_cible} m)",  # Nom de la couche
        style_function=lambda x, color=fill_color: {
            "fillColor": color,  # Couleur dynamique
            "color": "black",    # Couleur des contours
            "weight": 2,         # Épaisseur des contours
            "fillOpacity": 0.6   # Transparence
        },
        tooltip=folium.Tooltip(f"Massif : {nom_massif}<br>Épaisseur : {epaisseur_neige} cm"),  # Info-bulle
        show=True  # Affichage initial
    ).add_to(carte)

# Ajouter une légende
colormap.caption = "Épaisseur de neige (cm)"
colormap.add_to(carte)

# Ajouter un contrôle de couches
folium.LayerControl().add_to(carte)

# Sauvegarder la carte
carte.save("carte_epaisseur_neige_altitude.html")


NameError: name 'df' is not defined